In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


# Set Image Size & Batch Size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define dataset directory paths
dataset_dir = r"D:\01 STUDY MATERIAL\ai project\Breast-Splitted"
train_dir = f"{dataset_dir}/train"
val_dir = f"{dataset_dir}/val"
test_dir = f"{dataset_dir}/test"

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# Validation & Test: No augmentation, just rescaling
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load dataset using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary", shuffle=False
)

# Define CNN Model (Conv2D-based)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Helps prevent overfitting
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=5)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the trained model
# model.save("cnn_model_tf.h5")

Found 4745 images belonging to 2 classes.
Found 1581 images belonging to 2 classes.
Found 1583 images belonging to 2 classes.
Epoch 1/5
149/149 [==============================] - 215s 1s/step - loss: 0.7959 - accuracy: 0.7937 - val_loss: 1.7077 - val_accuracy: 0.6863
Epoch 2/5
  5/149 [>.............................] - ETA: 2:50 - loss: 0.4058 - accuracy: 0.8500